In [1]:
import numpy as np
import cv2
import torch
import imutils
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
prototxt = 'face_detector/deploy.prototxt'
weights = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'
cvnet = cv2.dnn.readNet(prototxt, weights)

In [23]:
net = torch.load('models/21052021_040217_resnet18_99.8779_epoch_33.pth', map_location=torch.device('cpu'))
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
image = cv2.imread('self.jpg')
h, w = image.shape[:2]
if h>1000:
    image = imutils.resize(image, height=1000)
elif w>1000:
    image = imutils.resize(image, width=1000)
ori = image.copy()
(h, w) = image.shape[:2]

In [13]:
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))

In [14]:
cvnet.setInput(blob)
detections = cvnet.forward()

In [37]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

In [ ]:
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7]*np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = transform(Image.fromarray(np.uint8(face))).unsqueeze(dim=0)
#         plt.imshow(face.permute(1,2,0))
        with torch.no_grad():
            out = net.forward(face)
        label = 'Mask' if out<0 else 'No Mask'
        color = (0, 255, 0) if label=='Mask' else (0, 0, 255)
#         label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
cv2.imshow("Output", image)
cv2.waitKey(0)